In [1]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import models
import encoders
import decoders
from training import train, test, checkpoint
from dataloaders import load_data, create_loaders

WANDB_API_KEY= '52c84ab3f3b5c1f999c7f5f389f5e423f46fc04a'
import wandb
wandb.login()

cuda


wandb: Currently logged in as: raraz15 (use `wandb login --relogin` to force relogin)


True

In [3]:
#dataset_path = '/projects/bassline_transcription/data/datasets'
#dataset_path = '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets'
dataset_path = '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]'

dataset_name = 'traxsource_0-5000_bassline_representations'
M = 8 


data_params = {'dataset_path': dataset_path,
               'dataset_name': dataset_name,
               'scale_type': 'min',
               'M': M,
               'batch_size': 64}

X = load_data(data_params)

X = append_SOS(X)

frequencies = np.unique(X, return_counts=True)

K = X.max()+1 # Number of classes, assumes consecutive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

Number of classes: 27
Sequence Length: 65
Number of data points: 3481


cross_entropy_weights = torch.FloatTensor(1 / (frequencies[1] / frequencies[1].max())).to(device)
print(cross_entropy_weights)

In [4]:
embedding_params = {'num_embeddings': K,
                    'embedding_size': 32}

encoder_params = {'num_embeddings': embedding_params['num_embeddings'],
                  'embedding_size': embedding_params['embedding_size'],
                  'hidden_size': 256,
                  'dropout': 0.0,  
                  'num_layers': 1,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

decoder_params = {'num_embeddings': embedding_params['num_embeddings'],
                  'embedding_size': embedding_params['embedding_size'],
                  'hidden_size': encoder_params['hidden_size'],
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0.0,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device,
                 'teacher_forcing_ratio': 1.0}

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 1000}

all_params = {'embedding_params':embedding_params,
              'encoder_params': encoder_params,
              'decoder_params':decoder_params, 
              'train_params':train_params}

In [5]:
encoder = encoders.Seq2SeqEncoder(**encoder_params).to(device)
decoder = decoders.Seq2SeqDecoder(**decoder_params).to(device)
model = models.Seq2Seq(encoder, decoder).to(device)

print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss(reduction='mean') #, weight=cross_entropy_weights
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_loader, validation_loader, test_loader = create_loaders(X, data_params)

Seq2Seq(
  (encoder): Seq2SeqEncoder(
    (embedding): Embedding(27, 32)
    (net): LSTMnetwork(
      (lstm): LSTM(32, 256, batch_first=True, bidirectional=True)
    )
  )
  (decoder): Seq2SeqDecoder(
    (embedding): Embedding(27, 32)
    (rnn): LSTMnetwork(
      (lstm): LSTM(288, 256, batch_first=True)
    )
    (linear): Linear(in_features=544, out_features=27, bias=True)
  )
)
Number of parameters: 1169467


In [ ]:
project_name = 'PyTorch_experiments'

model_name = dt.datetime.strftime(dt.datetime.now(),"%d_%m__%H_%M")

with wandb.init(project=project_name, name=model_name, config=all_params, entity='nbg'):

    test_loss, test_acc = test(model, test_loader, criterion, device)
    samples = model.sample()
    print('Test Loss Before Training: {:.6f}'.format(test_loss))
    print('Test Accuracy Before Training: {:.3f}'.format(test_acc))    
    print('Initial Sample:\n{}\n'.format(samples[0]))    
    wandb.log({'test_loss': test_loss, 'test_accuracy': test_acc, 'samples': samples})

    for epoch in tqdm(range(train_params['N_epochs'])):

        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        validation_loss, validation_acc = test(model, validation_loader, criterion, device)
        
        model.decoder.update_teacher_forcing_ratio(decoder_params['teacher_forcing_ratio']/train_params['N_epochs'])       
        
        wandb.log({'train_loss': train_loss, 'train_accuracy': train_acc,
                  'validation_loss': validation_loss, 'validation_acc': validation_acc})
        
        checkpoint(model_name, model, optimizer, validation_acc, epoch)

        if not (epoch % 25):
            print('Epoch: {}, train_loss: {:.6f}, train_acc: {:.3f}, \
            val_loss: {:.6f}, val_acc: {:.3f}'.format(epoch, train_loss, train_acc, validation_loss, validation_acc))
            
        if not (epoch % 50):
            samples = model.sample()
            print('Sample:\n{}'.format(samples[0]))
            wandb.log({'samples': samples})
            
    test_loss, test_acc = test(model, test_loader, criterion, device)
    samples = model.sample()
    print('\nTest Loss After Training: {:.6f}'.format(test_loss))
    print('Test Accuracy After Training: {:.3f}'.format(test_acc))
    print('\nFinal Sample:\n{}'.format(samples[0]))
    wandb.log({'test_loss': test_loss, 'test_accuracy': test_acc, 'samples': samples})    

  0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss Before Training: 3.248304
Test Accuracy Before Training: 0.000
Initial Sample:
tensor([22,  2, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3,
        22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3,
        22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22,  3,
        22,  3, 22,  3, 22,  3, 22,  3, 22,  3, 22], device='cuda:0')



  0%|          | 1/1000 [00:04<1:14:07,  4.45s/it]

Epoch: 0, train_loss: 1.363090, train_acc: 0.662,             val_loss: 0.582728, val_acc: 0.877
Sample:
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


  3%|▎         | 26/1000 [01:40<1:00:05,  3.70s/it]

...

 10%|█         | 101/1000 [06:22<54:57,  3.67s/it] 

Epoch: 100, train_loss: 0.137892, train_acc: 0.960,             val_loss: 0.124108, val_acc: 0.965
Sample:
tensor([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19], device='cuda:0')


 13%|█▎        | 126/1000 [07:58<57:04,  3.92s/it]

Epoch: 125, train_loss: 0.138533, train_acc: 0.959,             val_loss: 0.179749, val_acc: 0.949


 14%|█▍        | 140/1000 [08:52<56:05,  3.91s/it]

In [ ]:
test_loss, test_acc = test(model, test_loader, criterion, device)
samples = model.sample()
print('Test Loss Before Training: {:.6f}'.format(test_loss))
print('Test Accuracy Before Training: {:.3f}'.format(test_acc))    
print('Initial Sample:\n{}\n'.format(samples[0]))    

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    validation_loss, validation_acc = test(model, validation_loader, criterion, device)

    model.decoder.update_teacher_forcing_ratio(decoder_params['teacher_forcing_ratio']/train_params['N_epochs'])       

    checkpoint(model_name, model, optimizer, validation_acc, epoch)

    if not (epoch % 25):
        print('Epoch: {}, train_loss: {:.6f}, train_acc: {:.3f}, \
        val_loss: {:.6f}, val_acc: {:.3f}'.format(epoch, train_loss, train_acc, validation_loss, validation_acc))

    if not (epoch % 50):
        samples = model.sample()
        print('Sample:\n{}'.format(samples[0]))

test_loss, test_acc = test(model, test_loader, criterion, device)
samples = model.sample()
print('\nTest Loss After Training: {:.6f}'.format(test_loss))
print('Test Accuracy After Training: {:.3f}'.format(test_acc))
print('\nFinal Sample:\n{}'.format(samples[0])) 

In [ ]:
test_loss, test_acc = test(model, test_loader, criterion, device)
samples = model.sample()

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    validation_loss, validation_acc = test(model, validation_loader, criterion, device)
    
    model.decoder.update_teacher_forcing_ratio(decoder_params['teacher_forcing_ratio']/train_params['N_epochs'])
    
    if epoch+1 % 5:
        print('Epoch: {}, train_loss: {:.6f}, acc: {:.3f}'.format(epoch+1, train_loss, train_acc))

test_loss, test_acc = test(model, test_loader, criterion, device)
print('Test Loss After Training: {:.6f}'.format(test_loss))

embedding = nn.Embedding(**embedding_params)

encoder = encoders.StackedBidirLSTMEncoder(**encoder_params).to(device)

#decoder = decoders.StackedUnidirLSTMDecoder(**decoder_params).to(device)
decoder = decoders.StackedUnidirLSTMDenseDecoder(**decoder_params).to(device)

model = models.VanillaAutoEncoder(embedding, encoder, decoder).to(device)
print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss(reduction='mean') #, weight=cross_entropy_weights
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

encoder_params = {'num_embeddings': K,
                  'embedding_dim': 16, #??
                  'hidden_size': 128,
                  'dropout': 0,  
                  'num_layers': 4,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

decoder_params = {'input_size': encoder_params['hidden_size'],
                  'output_size': K,
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device}

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 10}

all_params = {'encoder_params': encoder_params, 'decoder_params':decoder_params, 'train_params':train_params}

embedding_params = {'num_embeddings': K,
                    'embedding_dim': 32}

encoder_params = {'input_size': embedding_params['embedding_dim'],
                  'hidden_size': embedding_params['embedding_dim'],
                  'dropout': 0.5,  
                  'num_layers': 16,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

#StackedUnidirLSTMDenseDecoder / StackedUnidirectionalLSTMDecoder
decoder_params = {'input_size': encoder_params['hidden_size'],
                  'output_size': K,
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0.5,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device,
                 'teacher_forcing_ratio': 1.0}

#StackedUnidirectionalLSTMDecoderwithEmbedding 
"""decoder_params = {'num_embeddings': K,
                  'embedding_dim': 16,
                  'hidden_size': 128,                  
                  'num_layers': encoder_params['num_layers'], 
                  'dropout': 0.2,
                  'batch_size': data_params['batch_size'],
                  'sequence_length': sequence_length,
                  'device':device}"""

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 500}

all_params = {'embedding_params':embedding_params,'encoder_params': encoder_params,
              'decoder_params':decoder_params, 'train_params':train_params}

encoder = encoders.StackedBidirLSTMEncoder(**encoder_params).to(device)

decoder = decoders.StackedUnidirLSTMDecoderwithEmbedding(**decoder_params).to(device)

model = models.AutoEncoder(encoder, decoder, device).to(device)
print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)